In [1]:
from torch import nn
import torch
import torch.nn as nn  
import torch.nn.functional as F  
import torch.utils.data as tud  
from torch.nn.parameter import Parameter  #参数更新和优化函数
from collections import Counter 
import numpy as np 
import random
import math 
import pandas as pd
import scipy  #
import sklearn
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度函数

# 负例采样就是Skip-Gram模型的输出不是周围词的概率了，是正例和负例的概率


USE_CUDA = torch.cuda.is_available()

random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)
 

K = 3   # 负样本随机采样数量
C = 1    # 周围单词的数量
NUM_EPOCHS = 2 
VOCAB_SIZE = 20 
BATCH_SIZE = 4
LEARNING_RATE = 0.2 
EMBEDDING_SIZE = 5
#对应的维度
    
LOG_FILE = "word-embedding.log"

In [2]:
def word_tokenize(text): 
    return text.split()


with open("text8.train.txt", "r") as file: 
    text = file.read() # 一次性读入文件所有内容为一个字符串
    
text = [w for w in word_tokenize(text.lower())] 
#词语对应的编号
vocab = dict(Counter(text).most_common(VOCAB_SIZE-1))
#词语出现的次数
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))
#目前这里面的["<unk>"]对应的内容是必为0的内容
print('text = ***')
print(text)
print('vocab = ***')
print(vocab)
idx_to_word = [word for word in vocab.keys()] 
word_to_idx = {word:i for i, word in enumerate(idx_to_word)}
#idx_to_word为所有切出来的单词构成的list
print('idx_to_word = ***')
print(idx_to_word)
print('word_to_idx = ***')
print(word_to_idx)
#word_to_idx为将单词进行相应的编号
word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
print('word_counts = ***')
print(word_counts)
#word_counts为所有对应的单词构成的相应的矩阵
#这里word_counts共有对应的483个单词
print(len(word_counts))
word_freqs = word_counts / np.sum(word_counts)
print('word_freqs1 = ###')
print(word_freqs)
word_freqs = word_freqs ** (3./4.)
#计算词频，按照原文转换为3/4次方
word_freqs = word_freqs / np.sum(word_freqs)  # 用来做 negative sampling
#词频为词频/词频的总和
print('word_freqs = ***')
print(word_freqs)
#将所有的单词个数转化为一个483长度的词频矩阵

text = ***
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of']
vocab = ***
{'of': 2, 'anarchism': 1, 'originated': 1, 'as': 1, 'a': 1, 'term': 1, 'abuse': 1, 'first': 1, 'used': 1, 'against': 1, 'early': 1, 'working': 1, 'class': 1, 'radicals': 1, 'including': 1, 'the': 1, 'diggers': 1, '<unk>': 0}
idx_to_word = ***
['of', 'anarchism', 'originated', 'as', 'a', 'term', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', '<unk>']
word_to_idx = ***
{'of': 0, 'anarchism': 1, 'originated': 2, 'as': 3, 'a': 4, 'term': 5, 'abuse': 6, 'first': 7, 'used': 8, 'against': 9, 'early': 10, 'working': 11, 'class': 12, 'radicals': 13, 'including': 14, 'the': 15, 'diggers': 16, '<unk>': 17}
word_counts = ***
[2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
18
word_freqs1 = ###
[0.11111111 0.05555556 0.05555556 0.05555556 0.05

In [3]:
# 实现Dataloader
class Dataset(tud.Dataset): # 继承tud.Dataset父类
    
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):    
        super(Dataset, self).__init__() 
        print('Dataset __init__')
        self.text_encoded = [word_to_idx.get(t, VOCAB_SIZE-1) for t in text]
        #依次对text当中的单词进行相应的查找
        # get()返回指定键的值，没有则返回默认值
        # 这里面的key -- 字典中要查找的键，default -- 如果指定键的值不存在时，返回该默认值。
        self.text_encoded = torch.Tensor(self.text_encoded).long()
        #变成tensor类型，这里变成longtensor，也可以torch.LongTensor
        self.word_to_idx = word_to_idx 
        self.idx_to_word = idx_to_word  
        self.word_freqs = torch.Tensor(word_freqs) 
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self): 
        return len(self.text_encoded) #所有单词的总数
        
    def __getitem__(self, idx):
        ''' 这个function返回以下数据用于训练
            - 中心词
            - 这个单词附近的(positive)单词
            - 随机采样的K个单词作为negative sample
        '''
        #print('Dataset__getitem__')
        #self.text_encoded为原来英文语句的内容
        center_word = self.text_encoded[idx] 
        #print('center_word = ***')
        #print(center_word.shape)
        pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        #取余，防止超出句子的范围(这里面把句子看成一个环形的句子)
        #print('pos_indices = ###')
        #print(pos_indices)
        pos_words = self.text_encoded[pos_indices]
        # replacement=True有放回的取
        #print('pos_words.shape[0] = ###')
        #print(pos_words.shape[0])
        # 这里面的pos_words.shape[0] = 6,为左边取3个词组以及右边取3个词组
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], replacement=True)
        #print('neg_words = ###')
        #print(neg_words)
        # torch.multinomal(input,num_samples,replacement=False,out=None)        
        #!!!replace=True为有放回的选取，这里K*pos_words.shape[0]为负采样的个数
        
        #这里使用multinomial的原因就在于随着样本频率的增加，它被抽样的概率越来越大
        #负采样当中如果样本频率增加的情况下被采样的概率越来越大
        
        #!!!高频词在负采样中容易被选出来 所以即使左边3个词组和右边3个词组
        
        #!!!左右各取3个数值之后，负采样的个数为6*10个
        #每次返回的时候center_word = [],pos_words = [6],neg_words = [60]
        return center_word, pos_words, neg_words 


dataset = Dataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)  
#BATCH_SIZE = 128,每一个批次随机取出128个对应的数值，这里打包成DataLoader的主要原因是
#便于每次取出一个相应的批次操作

Dataset __init__


In [4]:
# 定义PyTorch模型
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
    #放入的vocab_size=30000,embed_size=100
        super(EmbeddingModel, self).__init__()
        print('EmbeddingModel __init__')
        self.vocab_size = vocab_size  #30000
        self.embed_size = embed_size  #100
              
        # 模型输入，输出是两个一样的矩阵参数nn.Embedding(30000, 100)
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        #torch.nn.Embedding(num_embeddings:int,embedding_dim:int,sparse:False)
        #如果sparse=True的情况下出来的为稀疏矩阵
        #!!!embedding为一个简单的存储固定大小的词典的嵌入向量的查找表
        #给一个编号，返回这个编号对应的嵌入向量
        #torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, 
        #max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None)
        
        #这里面的num_embeddings=30000,embedding_dim=100,
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
         # 权重初始化的一种方法
        initrange = 0.5 / self.embed_size
        #因为embed_size = 100,所以initrange = 0.005
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        self.out_embed.weight.data.uniform_(-initrange, initrange)
        print('origin = ***')
        print('self.in_embed = ***')
        print(self.in_embed)
        print('self.out_embed = ***')
        print(self.out_embed)
        #因为输出的值为(1/2)/self.embed_size=0.5/self.embed_size,
        #所以self.in_embed的范围为(-initrange,initrange)即(-0.005,0.005)
        #self.out_embed的范围为(-initrange,initrange)
        
    def forward(self, input_labels, pos_labels, neg_labels):
        '''
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围出现过的单词 [batch_size * (c * 2)],左边找出c个词组，右边找出c个词组
        neg_labelss: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (c * 2 * K)]
        return: loss, [batch_size]
        '''
        print('EmbeddingModel forward')
        batch_size = input_labels.size(0) 
        print('input_labels = !!!')
        print(input_labels)
        print('pos_labels = !!!')
        print(pos_labels)
        print('neg_labels = !!!')
        print(neg_labels)
        print('batch_size = %d'%batch_size)
        #原先这里input_labels = [128]
        #pos_labels = [128,6],neg_labels = [128,60]
        
       
        input_embedding = self.in_embed(input_labels) # B * embed_size
        #这里的self.in_embed为定义过的nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        #一个相应的函数，self.vocab_size=30000,self.embed_size=100
        
        #输入: (∗) , 包含提取的编号的任意形状的长整型张量。
        #输出: (∗,H) , 其中 * 为输入的形状，H为embedding_dim
        #embedding_dim的选择要注意，根据自己的符号数量，举个例子，如果你的词典尺寸是1024，那么极限压缩
        #（用二进制表示）也需要10维，再考虑词性之间的相关性，怎么也要在15-20维左右，虽然embedding是用来降维的，
        #但是也要注意这种极限维度，结合实际情况，合理定义
        
        #扩展维度之后生成的相当于一个随机矩阵的内容
        
        #经过self.in_embed(input_labels)处理之后input_embedding=[128,100]
        #pos_labels = [128,6,100],neg_labels = [128,60,100]
        pos_embedding = self.out_embed(pos_labels) # B * (2C) * embed_size 
        neg_embedding = self.out_embed(neg_labels) # B * (2*C*K) * embed_size
        #!!!注意这里面使用到了对应的out_embed相应的参数
        print('@@@input_embedding = @@@')
        print(input_embedding)
        print('@@@pos_embedding = @@@')
        print(pos_embedding)
        print('@@@neg_embedding = @@@')
        print(neg_embedding)
        #input_labels:[batch_size]
        #pos_labels:[batch_size,(windows_size*2)]
        #neg_labels:[batch_size,(windows_size*2*k)]
        #torch.bmm()为batch间的矩阵相乘（b,n.m)*(b,m,p)=(b,n,p)
        print('input_embedding.unsqueeze(2) = ***')
        print(input_embedding.unsqueeze(2))
        #input_embedding = [128,100],input_embedding.unsqueeze(2) = [128,100,1]
        #pos_embedding = [128,6,100]
        log_pos = torch.bmm(pos_embedding, input_embedding.unsqueeze(2)) # B * (2*C)
        #input_embedding.unsqueeze(2) = [batch_size,embedding_size,1]

        print('@@@log_pos = @@@')
        print(log_pos)
        #[128,6,100]*[128,100,1] = [128,6,1]
        #[batch_size,window_size*2,embedded]*[batch_size,embeded,1] = [batch_size,window_size*2,1]
        log_pos = log_pos.squeeze()
        print('@@@log_pos.squeeze() == ***')
        #squeeze()将为1的维度缩出来，所以log_pos = [128,6]
        print(log_pos)
        log_neg = torch.bmm(neg_embedding, -input_embedding.unsqueeze(2)).squeeze() # B * (2*C*K)
        #neg_embedding = [128,60,100],input_embedding.unsqueeze(2) = [128,100,1]
        #相乘之后的结果为[128,60,1]
        print('@@@log_neg = @@@')
        print(log_neg)
        print('@@@log_pos = @@@')
        print(log_pos)
        #下面loss计算就是论文里的公式
        #log_neg.shape = [128,60]
        #log_pos.shape = [128,6]
        
        log_pos = F.logsigmoid(log_pos).sum(1) # [batch_size]
        log_neg = F.logsigmoid(log_neg).sum(1) # [batch_size]   
        print('######################log_pos = #######################')
        print(log_pos)
        print('######################log_neg = #######################')
        print(log_neg)
        loss = log_pos + log_neg  # 正样本损失和负样本损失和尽量最大
        #如果为负数的时候就是损失和尽量最小
        #对应的大小为[batch_size]
        #因为需要提取出来的这128个维度的单词集体操作
        return -loss 
        #注意这里return的是-loss，最终的optimizer.step中还带有一个减号
        #所以这里如果是当前选中的这个单词的周边单词的话的128个维度单词的梯度被减去，而周边单词的梯度被加上，
        #而如果是这128个单词负采样的话这128个维度的单词
    
    # 模型训练有两个矩阵，self.in_embed和self.out_embed两个, 作者认为输入矩阵比较好，舍弃了输出矩阵
    # 取出输入矩阵参数，self.in_embed的矩阵为正采样的相应的矩阵，self.out_embed为负采样的相应矩阵
    def input_embeddings(self):   
        return self.in_embed.weight.data.cpu().numpy() 
    def output_embeddings(self):
        return self.out_embed.weight.data.cpu().numpy()


In [5]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
#VOCAB_SIZE = 30000,EMBEDDING_SIZE = 100
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
#model = model.cuda(),这里面的LEARING_RATE=0.2

EmbeddingModel __init__
origin = ***
self.in_embed = ***
Embedding(20, 5)
self.out_embed = ***
Embedding(20, 5)



# 寻找nearest neighbors
~~~
def find_nearest(word):
    '''embedding_weights是一个[vocab_size, embedding_size]的参数矩阵'''
    index = word_to_idx[word] 
    embedding = embedding_weights[index] # 取出这个单词的embedding向量
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    # 计算所有30000个embedding向量与传入单词embedding向量的相似度距离
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]] # 返回前10个最相似的
~~~

In [6]:
for e in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        #这里的enumerate(dataloader)使用__getitem__(self,idx)提取对应的值
        #!!!注意这里是dataloader，而不是对应的dataset数组
        #这里面是按照batch提取的，一次提取多个batch，这里的batch应该是打乱顺序进行随机抽取的
        
        #本来dataset中的数值的形状为([],[6],[60])
        #经过dataloader处理之后的对应数值为([128],[128,6],[128,60])
        #(打乱顺序随机取出对应的128个数值)
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        print('input_labels = !!!')
        print(input_labels)
        print('pos_labels = !!!')
        print(pos_labels)
        print('neg_labels = !!!')
        print(neg_labels)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!before backward and step!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(model.input_embeddings())
        print('----------------------------------------------------------------------------------------------------------------')
        print(model.output_embeddings())
        optimizer.zero_grad()
        #!!!注意每次optimizer需要将梯度降为0
        loss = model(input_labels, pos_labels, neg_labels).mean() 
        # model返回的是一个batch所有样本的损失，需要求个平均
        #这里求.mean()损失函数的平均的时候
        print('loss = !!!')
        print(loss)
        print('#################################after backward and step#######################################################')
        print(model.input_embeddings())
        print('---------------------------------------------------------------------------------------------------------------')
        print(model.output_embeddings())
        loss.backward()
        optimizer.step()
        #注意这里w不是直接减去ita*loss
        #而是w减去ita*(损失函数对自变量的导)
        #这里有个反向求导的对应的过程loss.backward(),反向求导之后再进行相应的
        #优化
        #出现次数较多的关联项下降的比较快，出席次数较少的关联项下降的比较慢
        #本质上就是当前这个对应的变量上升相应的梯度，其他变量下降相应的梯度
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@after backward and step@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        print(model.input_embeddings())
        print('--------------------------------------------------------------------------------------------------------------')
        print(model.output_embeddings())
        #函数之中定义了对应的input_embeddings()的内容为in_embed的相应的内容
        if i % 100 == 0:
            with open(LOG_FILE, "a") as fout: 
                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))
        
        #if i % 2000 == 0: 
        #    embedding_weights = model.input_embeddings()  # 取出训练中的in_embed词向量
            # 在三个词文本上评估词向量
        #    sim_simlex = evaluate("simlex-999.txt", embedding_weights)
        #    sim_men = evaluate("men.txt", embedding_weights)
        #    sim_353 = evaluate("wordsim353.csv", embedding_weights)
            
            #with open(LOG_FILE, "a") as fout:
            #    print("epoch: {}, iter: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
            #        e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
            #    fout.write("epoch: {}, iter: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
            #        e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                
    embedding_weights = model.input_embeddings() # 调用最终训练好的embeding词向量
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights) # 保存参数
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE)) # 保存参数

input_labels = !!!
tensor([11, 14,  3, 16])
pos_labels = !!!
tensor([[10, 12],
        [13, 15],
        [ 2,  4],
        [15,  0]])
neg_labels = !!!
tensor([[ 9, 16,  4,  8,  7, 14],
        [ 6, 13,  8,  8, 15,  2],
        [ 1,  2, 14,  5, 16, 11],
        [ 4, 14,  0, 11,  1, 10]])
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!before backward and step!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[[-0.09634085 -0.05079702 -0.09764721 -0.05053148  0.02803528]
 [ 0.07910234 -0.00253747 -0.0092114   0.08016897  0.07994234]
 [-0.00411206  0.07705157  0.07703517 -0.07001527  0.01060919]
 [-0.04474556  0.09511333 -0.04521798 -0.04240331  0.0406464 ]
 [ 0.0812582  -0.05959148  0.09809767 -0.02271371 -0.00155157]
 [-0.0691928   0.05849885  0.07399582  0.00072781  0.01321235]
 [ 0.01627956  0.08347691 -0.03223564 -0.01795088 -0.03631908]
 [ 0.08655737 -0.00060815  0.09565913  0.0751775  -0.05534681]
 [-0.09734754 -0.09383321 -0.01098482 -0.05939215  0.0826202 ]
 [ 0.0777399   0.08806259 -0.074

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@after backward and step@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[[-0.09888365 -0.05281906 -0.09474792 -0.06153099  0.0207892 ]
 [ 0.07659252  0.00032632 -0.01152079  0.08793088  0.08579083]
 [-0.00375843  0.0756731   0.08186354 -0.07156494  0.00854443]
 [-0.0433238   0.10042502 -0.04139317 -0.04680085  0.0368204 ]
 [ 0.08044905 -0.05788083  0.0971237  -0.02681437  0.0003005 ]
 [-0.0691928   0.05849885  0.07399582  0.00072781  0.01321235]
 [ 0.01627956  0.08347691 -0.03223564 -0.01795088 -0.03631908]
 [ 0.08655737 -0.00060815  0.09565913  0.0751775  -0.05534681]
 [-0.09838018 -0.08609484 -0.00992148 -0.06808605  0.07892119]
 [ 0.08154672  0.08782614 -0.08089297 -0.03173636  0.00757106]
 [-0.08350182  0.0179262  -0.00343599 -0.0780361  -0.10408156]
 [-0.0349428   0.02440172 -0.06975438 -0.08209933  0.03363008]
 [ 0.09942765 -0.04361018 -0.04975622 -0.03407041  0.09263287]
 [ 0.09879393  0.07071277 -0.05220384 -0.08503453  0.09695413]
 [ 0.01

tensor([[[-0.0074],
         [ 0.0053]],

        [[ 0.0098],
         [ 0.0127]],

        [[ 0.0071],
         [-0.0005]],

        [[-0.0061],
         [-0.0055]]], grad_fn=<BmmBackward>)
@@@log_pos.squeeze() == ***
tensor([[-0.0074,  0.0053],
        [ 0.0098,  0.0127],
        [ 0.0071, -0.0005],
        [-0.0061, -0.0055]], grad_fn=<SqueezeBackward0>)
@@@log_neg = @@@
tensor([[-0.0078, -0.0053,  0.0044,  0.0091, -0.0039, -0.0039],
        [ 0.0006,  0.0006, -0.0127,  0.0080, -0.0218, -0.0014],
        [-0.0024,  0.0033,  0.0025, -0.0038, -0.0055,  0.0032],
        [-0.0018,  0.0020, -0.0094, -0.0072, -0.0068, -0.0072]],
       grad_fn=<SqueezeBackward0>)
@@@log_pos = @@@
tensor([[-0.0074,  0.0053],
        [ 0.0098,  0.0127],
        [ 0.0071, -0.0005],
        [-0.0061, -0.0055]], grad_fn=<SqueezeBackward0>)
######################log_pos = #######################
tensor([-1.3873, -1.3751, -1.3830, -1.3921], grad_fn=<SumBackward1>)
######################log_neg = ################

######################log_pos = #######################
tensor([-1.3857, -1.3758, -1.3780, -1.3891], grad_fn=<SumBackward1>)
######################log_neg = #######################
tensor([-4.1584, -4.1657, -4.1747, -4.1672], grad_fn=<SumBackward1>)
loss = !!!
tensor(5.5486, grad_fn=<MeanBackward0>)
#################################after backward and step#######################################################
[[-0.10725258 -0.0482505  -0.08675849 -0.06216525  0.00629544]
 [ 0.07364302  0.0022982  -0.01059497  0.089246    0.08864496]
 [-0.00802662  0.07961414  0.08324979 -0.06912722  0.00171028]
 [-0.04294299  0.09924994 -0.03610822 -0.04722053  0.0394378 ]
 [ 0.074784   -0.04972069  0.09695041 -0.02280641  0.00028184]
 [-0.07956024  0.06849203  0.0847001   0.00531577  0.00089992]
 [ 0.01468543  0.08337899 -0.03046825 -0.0272178  -0.025835  ]
 [ 0.08339272  0.00374357  0.10135104  0.06951786 -0.06610727]
 [-0.09838018 -0.08609484 -0.00992148 -0.06808605  0.07892119]
 [ 0.08154672  0.087

In [7]:
print(embedding_weights.shape)

(20, 5)


In [8]:
print(embedding_weights)

[[-0.10725258 -0.0482505  -0.08675849 -0.06216525  0.00629544]
 [ 0.07364302  0.0022982  -0.01059497  0.089246    0.08864496]
 [-0.00802662  0.07961414  0.08324979 -0.06912722  0.00171028]
 [-0.04294299  0.09924994 -0.03610822 -0.04722053  0.0394378 ]
 [ 0.074784   -0.04972069  0.09695041 -0.02280641  0.00028184]
 [-0.07956024  0.06849203  0.0847001   0.00531577  0.00089992]
 [ 0.01468543  0.08337899 -0.03046825 -0.0272178  -0.025835  ]
 [ 0.08365669  0.0064706   0.10129436  0.05965371 -0.06762496]
 [-0.09551293 -0.08569594 -0.00187606 -0.07653442  0.08757468]
 [ 0.08310555  0.08373493 -0.08451287 -0.03436622  0.01233155]
 [-0.08597033  0.02047984  0.00065999 -0.07752654 -0.10261613]
 [-0.03025249  0.02512149 -0.07240909 -0.08607884  0.02911239]
 [ 0.09610046 -0.04913365 -0.03815091 -0.03238004  0.09234068]
 [ 0.10047776  0.06891955 -0.06032153 -0.08875097  0.10059984]
 [ 0.00940675 -0.03032099 -0.0385595   0.08759516 -0.02060052]
 [-0.03982215 -0.02724058 -0.01811096  0.08047485 -0.05

word_to_idx为单词对应的编号，embedding_weight为单词对应编号的相应的矩阵

 计算两个文本对应的余弦向量的值
 from sklearn.metrics.pairwise import cosine_similarity #余弦相似度函数
 word1_idx, word2_idx = word_to_idx[word1], word_to_idx[word2]
 word1_embed, word2_embed = embedding_weights[[word1_idx]], embedding_weights[[word2_idx]]
 model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed, word2_embed)))
  human_similarity.append(float(data.iloc[i, 2]))
            # 这个是人类统计得到的相似度
  return scipy.stats.spearmanr(human_similarity, model_similarity)# , model_similarity

如果判断两个句子是否相同，可以用model_similarity中的所有数平方加和得到最终的内容